<a href="https://colab.research.google.com/github/werlang/emolearn-ml-model/blob/main/time_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.engine import  Model
from keras.models import Sequential, load_model
from keras.layers import TimeDistributed, GRU, LSTM, Dropout, Conv2D, Conv3D, BatchNormalization, MaxPooling2D, MaxPooling3D, Flatten, Dense, Input, Add, Activation, AveragePooling3D, AveragePooling2D, Bidirectional, Concatenate
from keras.utils import Sequence, plot_model, to_categorical
from keras.optimizers import Adam, SGD
from keras.callbacks import TensorBoard, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping, Callback, ModelCheckpoint
from keras.metrics import AUC
from keras.regularizers import l2
import pandas as pd
import numpy as np
import os, cv2
import datetime
import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, recall_score, precision_score, confusion_matrix
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from keras import backend as K
import math
from IPython.display import Image
from numba import cuda
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler 


def restart():
    cuda.select_device(0)
    cuda.close()


def start_colab():
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    !pip install Keras-Applications


def extract_data():
    !mkdir features
    !mkdir of_features

    # aligned faces extracted from openface
    print("COPYING TRAIN SET...")
    !unzip -n -q "drive/My Drive/Doutorado/Implementação/daisee_aligned/Train_112.zip" -d features 
    print("COPYING VALIDATION SET...")
    !unzip -n -q "drive/My Drive/Doutorado/Implementação/daisee_aligned/Validation_112.zip" -d features 
    print("COPYING TEST SET...")
    !unzip -n -q "drive/My Drive/Doutorado/Implementação/daisee_aligned/Test_112.zip" -d features 
    print("COPYING OPENFACE FEATURES...")
    !unzip -n -q "drive/My Drive/Doutorado/Implementação/daisee_openface.zip" -d of_features
    print("COPYING LABELS...")
    !cp -r "drive/My Drive/Doutorado/Implementação/labels" ./
    # print("COPYING FOLDS...")
    # !unzip -n -q "drive/My Drive/1NOSYNC/DT/folds_engagement.zip" -d ./folds

    print("DONE")


def create_folds():
    print("CREATING FOLDS...")
    subs = {}
    Y = []
    clips = []
    for split in ["Train", "Validation", "Test"]:
        csv_path = "{}/{}Labels.csv".format(labels_path, split)
        csv = pd.read_csv(csv_path)
        C = np.array(csv['ClipID'])
        Yt = np.array(csv[emotion])
        if len(Y) == 0:
            clips = C
            Y = Yt
        else:
            Y = np.concatenate((Y, Yt), axis=0)
            clips = np.concatenate((clips, C), axis=0)

    # transform 0-3 to 0-1
    Y = to_categorical((np.floor(Y/2)), 2)
    # Y = to_categorical(Y, 5)

    # print(Y.shape)
    # print(np.unique(np.argmax(Y, axis=1), return_counts=True))

    for i in range(len(clips)):
        file_name = clips[i].split(".")[0]
        subject = int(file_name[:6])
        if not subject in subs:
            # subs[subject] = [0,0,0,0,0]
            subs[subject] = [0,0]
        subs[subject] = np.add(subs[subject], Y[i])

    # ratio = sum(Y[:,0]) / len(Y)
    # print(subs)

    folds = []
    fold_names = []
    for i in subs:
        if len(folds) < n_folds:
            folds.append(subs[i])
            fold_names.append([i])
        else:
            summed = np.sum(folds, axis=1)
            index = list(summed).index(min(summed))
            folds[index] = np.add(folds[index], subs[i])
            fold_names[index].append(i)

    # print(folds)
    # print(fold_names)
    # print(np.sum(folds, axis=1))

    print("GETTING FOLDS DATA...")
    # build frames array
    videos, opface, labels = [], [], []
    for i in range(len(fold_names)):
        videos.append([])
        opface.append([])
        labels.append([])

    fps = 15
    skip = int(round(interval * fps / time_frames, 0))

    for split in ["Train", "Validation", "Test"]:
        csv_path = "{}/{}Labels.csv".format(labels_path, split)
        csv = pd.read_csv(csv_path)
        clips = csv['ClipID']
        Y = np.floor(csv[emotion]/2)

        for Yi in range(len(clips)):
            file_name = clips[Yi].split(".")[0]
            subject = file_name[:6]

            for f in range(n_folds):
                if int(subject) in fold_names[f] and Y[Yi] <= 3:
                    # print("{}/{}".format(split, subject))
                    # get aligned faces file names into files array
                    dir_path = "{}/{}/{}/{}_aligned".format(features_path, split, subject, file_name)
                    of_path = "of_{}/{}/{}/{}.csv".format(features_path, split, subject, file_name)
                    files, filesof = [], []

                    if os.path.isfile(of_path):
                        csv_of = pd.read_csv(of_path)
                        # rows = csv_of.iloc[:,5:]
                        rows = csv_of[[' gaze_0_x',' gaze_0_y',' gaze_0_z',' gaze_1_x',' gaze_1_y',' gaze_1_z',' gaze_angle_x',' gaze_angle_y',' pose_Tx',' pose_Ty',' pose_Tz',' pose_Rx',' pose_Ry',' pose_Rz',' AU01_r',' AU02_r',' AU04_r',' AU05_r',' AU06_r',' AU07_r',' AU09_r',' AU10_r',' AU12_r',' AU14_r',' AU15_r',' AU17_r',' AU20_r',' AU23_r',' AU25_r',' AU26_r',' AU45_r',' AU01_c',' AU02_c',' AU04_c',' AU05_c',' AU06_c',' AU07_c',' AU09_c',' AU10_c',' AU12_c',' AU14_c',' AU15_c',' AU17_c',' AU20_c',' AU23_c',' AU25_c',' AU26_c',' AU28_c',' AU45_c']]
                        min_max_scaler = MinMaxScaler()
                        rows = min_max_scaler.fit_transform(rows)
                    
                    for i in range(1,350):
                        image_path = "{}/frame_det_00_000{:03d}.jpg".format(dir_path, i)
                        if os.path.isfile(image_path) and os.path.isfile(of_path):
                            files.append(image_path)
                            filesof.append(rows[i-1])

                    # append only the parts relative to the video section
                    for i in range(0, (len(files) - time_frames*skip) // skip + 1, np.max([1, int(stride * time_frames)])):
                        temp, tpof = [], []
                        for j in range(i*skip, (i+time_frames)*skip, skip):
                            temp.append(files[j])
                            tpof.append(filesof[j])
                        videos[f].append(temp)
                        opface[f].append(tpof)
                        labels[f].append(Y[Yi])

    return videos, opface, labels


def save_folds(videos, opface, labels):
    !mkdir folds

    first_only = True

    rus = RandomUnderSampler(random_state=42)
    ros = RandomOverSampler(random_state=42, sampling_strategy=0.13)
    for i in range(n_folds):
        print("SAVING FOLD {}...".format(i+1))
        train_X = videos.copy()
        train_Y = labels.copy()

        test_X = train_X.pop(i)
        test_Y = train_Y.pop(i)

        train_X = np.concatenate(train_X)
        train_Y = np.concatenate(train_Y)

        train_X, train_Y = ros.fit_resample(train_X, train_Y)
        train_X, train_Y = rus.fit_resample(train_X, train_Y)

        train_XO = opface.copy()
        train_YO = labels.copy()

        test_XO = train_XO.pop(i)
        test_YO = train_YO.pop(i)

        train_XO = np.concatenate(train_XO)
        train_YO = np.concatenate(train_YO)

        train_XO, train_YO = ros.fit_resample(train_XO.reshape(train_XO.shape[0], -1), train_YO)
        train_XO, train_YO = rus.fit_resample(train_XO, train_YO)
        train_XO = train_XO.reshape(train_XO.shape[0], time_frames, -1)

        # save folds to file
        train_Y = to_categorical(train_Y, 2)
        np.save("folds/fold_{}_train_Y.npy".format(i), train_Y)
        
        train_X = np.array(train_X)
        np.save("folds/fold_{}_train_XV.npy".format(i), train_X)

        train_XO = np.array(train_XO)
        np.save("folds/fold_{}_train_XO.npy".format(i), train_XO)

        test_Y = to_categorical(test_Y, 2)
        np.save("folds/fold_{}_test_Y.npy".format(i), test_Y)
        
        test_X = np.array(test_X)
        np.save("folds/fold_{}_test_XV.npy".format(i), test_X)

        test_XO = np.array(test_XO)
        np.save("folds/fold_{}_test_XO.npy".format(i), test_XO)

        if first_only:
            break


class Generator_V(Sequence):
    def __init__(self, gen_split, batch_size, frames):
        print("BUILDING GENERATOR...")
        self.batch_size = batch_size 
        self.labels, self.videos, = [], []

        self.videos = np.load("folds/fold_{}_{}_XV.npy".format(fold_step-1, gen_split.lower()))
        self.opface = np.load("folds/fold_{}_{}_XO.npy".format(fold_step-1, gen_split.lower()))
        self.labels = np.load("folds/fold_{}_{}_Y.npy".format(fold_step-1, gen_split.lower()))

        self.videos, self.opface, self.labels = shuffle(self.videos, self.opface, self.labels, random_state=0)
        # Y = np.array(self.labels)
        # X = np.array(self.videos)
        # for i in range(Y.shape[-1]):
        #     print("{}: {} ({:.2f}), ".format(i, sum(Y[:,i]), sum(Y[:,i]) / len(Y) * 100), end="")
        # print("")
        # print(X.shape)
        # np.save("{}/{}.npy".format(labels_path, gen_split), self.labels)
        print("Done")


    def __len__(self):
        return int(np.ceil(len(self.videos) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_xv = self.videos[idx * self.batch_size : (idx + 1) * self.batch_size]
        batch_xo = self.opface[idx * self.batch_size : (idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size : (idx + 1) * self.batch_size]

        # print(batch_x)
        videos, opface = [], []
        for video in batch_xv:
            images = []
            for name in video:
                img = cv2.imread(name, 0)
                images.append(img)
            videos.append(np.array(images)/255)

        for features in batch_xo:
            opface.append(features)
            
        videos = np.array(videos)
        opface = np.array(opface)
        label = np.array(batch_y)

        return [videos, opface], label


def avgacc(y_true, y_pred):
    def get_class_acc(id):
        class_id_true = K.argmax(y_true, axis=-1)
        class_id_pred = K.argmax(y_pred, axis=-1)
        # Replace class_id_preds with class_id_true for recall here
        accuracy_mask = K.cast(K.equal(class_id_pred, id), 'int32')
        class_acc_tensor = K.cast(K.equal(class_id_true, class_id_pred), 'int32') * accuracy_mask
        class_acc = K.sum(class_acc_tensor) / K.maximum(K.sum(accuracy_mask), 1)
        return class_acc

    return sum([get_class_acc(0), get_class_acc(1)]) / 2


def build_model():

    def model_video():
        def residual_block(input, output_size, short):
            # KR = None
            input_size = input.shape[-1]
            
            s = 1 if input_size == output_size else 2
            x = Conv2D(output_size//4, strides=s, kernel_regularizer=KR, kernel_size=1, padding='valid')(input)
            x = BatchNormalization()(x)
            x = Activation(act_func)(x)

            if not short:

                x = Conv2D(output_size//4, strides=1, kernel_regularizer=KR, kernel_size=3, padding='same')(x)
                x = BatchNormalization()(x)
                x = Activation(act_func)(x)

            x = Conv2D(output_size, strides=1, kernel_regularizer=KR, kernel_size=1, padding='valid')(x)
            x = BatchNormalization()(x)

            if input_size == output_size:
                shortcut = input
            else:
                shortcut = Conv2D(output_size, strides=2, kernel_regularizer=KR, kernel_size=1, padding='valid')(input)
                shortcut = BatchNormalization()(shortcut)

            x = Add()([x, shortcut])
            x = Activation(act_func)(x)

            return x

        KR = l2(1e-2)
        # KR = None
        act_func = 'relu'
        filters = 16

        inputImg = Input(shape=(112, 112, 1))
        inputVideo = Input(shape=(time_frames, 112, 112, 1))

        # first model
        x = Conv2D(filters, kernel_size=3)(inputImg)
        x = BatchNormalization()(x)
        x = Activation(act_func)(x)
        x = MaxPooling2D(pool_size=2)(x)
        
        blocks = [2, 3]
        for b in blocks:
            filters = filters * 2
            for i in range(b):
                x = residual_block(x, filters, True)

        x = AveragePooling2D(pool_size=2)(x)
        x = Flatten()(x)

        imgModel = Model(inputImg, x)

        x = TimeDistributed(imgModel)(inputVideo)

        model = Model(inputVideo, x)

        return model

    def model_openface():
        regularizer_value = 1e-2

        input = Input(shape=(time_frames, gen_train.__getitem__(0)[0][1].shape[2]))

        x = Bidirectional(LSTM(16, kernel_regularizer=l2(regularizer_value), recurrent_regularizer=l2(regularizer_value), return_sequences=True))(input)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        def residual_block(input, output_size):
            input_size = input.shape[-1] * 2
            
            x = Bidirectional(LSTM(output_size, kernel_regularizer=l2(regularizer_value), recurrent_regularizer=l2(regularizer_value), return_sequences=True))(input)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Bidirectional(LSTM(output_size, kernel_regularizer=l2(regularizer_value), recurrent_regularizer=l2(regularizer_value), return_sequences=True))(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Bidirectional(LSTM(output_size, kernel_regularizer=l2(regularizer_value), recurrent_regularizer=l2(regularizer_value), return_sequences=True))(x)
            x = BatchNormalization()(x)

            if input_size == output_size:
                shortcut = input
            else:
                shortcut = Bidirectional(LSTM(output_size, kernel_regularizer=l2(regularizer_value), recurrent_regularizer=l2(regularizer_value), return_sequences=True))(input)
                shortcut = BatchNormalization()(shortcut)

            x = Add()([shortcut,x])
            x = Activation('relu')(x)

            return x

        blocks = [2,2]
        filters = 16
        for b in blocks:
            filters = filters * 2
            for i in range(b):
                x = residual_block(x, filters)

        x = Bidirectional(LSTM(64, kernel_regularizer=l2(regularizer_value), recurrent_regularizer=l2(regularizer_value), return_sequences=True))(x)
        
        model = Model(input, x)

        return model
        
    print("Building model...")
    ########### IF BUILT, MUST DEFINE A NAME TO APPEND TO DIRECTORY NAME ###############
    global ident_name

    ########### IF LOADED, MUST DEFINE DIR NAME AND STARTING EPOCH ############
    global dir_name

    model_v = model_video()
    model_o = model_openface()

    x = Concatenate()([model_v.output, model_o.output])
    x = Bidirectional(LSTM(256, return_sequences=False))(x)

    # joint model
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.8)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.8)(x)
    x = Dense(2, activation='softmax')(x)

    model = Model([model_v.inputs, model_o.inputs], x)

    model.compile(
        loss='categorical_crossentropy',
        # optimizer = SGD(lr=0.1, decay=1e-6, momentum=0.9), 
        optimizer = Adam(lr=1e-4),
        # metrics=[avgacc,'accuracy'])
        metrics=['accuracy'])

    plot_model(model, show_layer_names=False, show_shapes=True)

    loadModel = False if epoch == 0 else True

    if not loadModel:
        t = datetime.datetime.now()
        prefix = str(t.year) +'-'+ str(t.month) +'-'+ str(t.day) +'-'+ str(t.hour) +'-'+ str(t.minute) +'-'+ str(t.second)
        save_dir = "{}/{}-{}".format(drive_save_path, prefix, ident_name)
        os.mkdir(save_dir)
    else:
        file_name = "{:03d}.h5".format(epoch)
        save_dir = "{}/{}".format(drive_save_path, dir_name)
        print("Loading model from {}/{}.".format(save_dir, file_name))
        model = load_model("{}/{}".format(save_dir, file_name), custom_objects={'avgacc': avgacc})

    return model, save_dir, epoch


def set_callbacks():
    #callbacks
    checkpoint = ModelCheckpoint(
        filepath = save_dir + '/{epoch:03d}.h5', 
        monitor = 'val_loss', 
        verbose=1, 
        save_best_only=True,
    )

    tensorboard = TensorBoard(
    	log_dir         = "{}/logs".format(save_dir),
    	histogram_freq  = 0,
    	write_graph     = True,
    	write_grads     = False,
    	write_images    = True
    )

    early_stop = EarlyStopping(
        monitor 	= 'val_loss',
        patience 	= 10,
        restore_best_weights = True,
        verbose     = 1,
        min_delta   = 1e-5
    )

    reduce_lr_plateau = ReduceLROnPlateau(
        monitor 	= 'val_loss',
        factor		= 0.2,
        patience	= 5,
        min_lr		= 1e-6,
        verbose     = 1
    )

    return [checkpoint, early_stop, reduce_lr_plateau]


def fit_model(train_weights):
    #calculate weights based on train set distribution
    label_array = np.load("folds/fold_{}_train_Y.npy".format(fold_step-1))
    weights = {0: 1, 1: 1}
    if train_weights != [1,1]:
        weights = {0: train_weights[0], 1: train_weights[1]}
        print("Train weights: {}".format(weights))

    def run():
        #run the model
        return model.fit(
            gen_train,
            epochs = 1000,
            validation_data = gen_test,
            # class_weight = weights,
            callbacks = callbacks,
            initial_epoch = epoch)
        
    hist = run()  

    plt.plot(hist.history['loss'], '#0000ff', label="loss")
    plt.plot(hist.history['val_loss'], '#ff0000', label="test_loss")
    plt.legend()
    plt.show()

    plt.plot(hist.history['loss'], '#0000ff', label="loss")
    plt.plot(hist.history['val_loss'], '#ff0000', label="test_loss")
    plt.axis([5, None, 0, None])
    plt.legend()
    plt.show()

    plt.plot(hist.history['accuracy'], '#0000ff', label="acc")
    plt.plot(hist.history['val_accuracy'], '#ff0000', label="test_acc")
    # plt.plot(hist.history['accuracy'], '#0055aa', label="acc")
    # plt.plot(hist.history['val_accuracy'], '#aa5500', label="val_acc")
    plt.legend()
    plt.show()


def predict(**kw):
    global gen_test
    try:
        gen_test
    except:
        gen_test = Generator_V("Test", batch_size, time_frames)
    
    if (kw and kw['force']):
        gen_test = Generator_V("Test", batch_size, time_frames)

    Y_true = np.load("folds/fold_{}_test_Y.npy".format(fold_step-1)).argmax(axis=1)

    print("Predicting...")
    Y_pred = (model.predict(gen_test, verbose=1)).argmax(axis=1)
    print("\nConfusion matrix:")
    cm = confusion_matrix(Y_true, Y_pred)
    print(cm)

    print("\nEvaluating...")
    ev = model.evaluate(gen_test, verbose=1)
    # print("Loss: {}, Acc: {}".format(ev[0], ev[1]))

    hits = [0,0]
    total = [0,0]
    for i in range(len(Y_true)):
        for c in range(2):
            if (Y_true[i] == c and Y_pred[i] == c):
                hits[c] = hits[c] + 1
            if Y_true[i] == c:
                total[c] = total[c] + 1

    print("\nPer class accuracy:")
    for c in range(2):
        hits[c] = hits[c] / total[c]
        print("{}: {}".format(c, hits[c]))

    print("Average class accuracy: {}".format(sum(hits) / len(hits)))
    return gen_test
        

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
emotion = "Engagement"
labels_path = "labels"
features_path = "features"
drive_save_path = 'drive/My Drive/1NOSYNC/DT/checkpoint'
ident_name = 'fusion-1e'
dir_name = '2020-12-2-3-9-46-conv3d-4em'
batch_size = 16
time_frames = 75
interval = 5
stride = 0.5
fold_step = 1
n_folds = 10

epoch = 0

# restart()
# start_colab()
# extract_data()
# v, o, l = create_folds()
# save_folds(v,o,l)

# gen_train = Generator_V('Train', batch_size, time_frames)
# gen_test = Generator_V('Test', batch_size, time_frames)
# model, save_dir, epoch = build_model()
# callbacks = set_callbacks()
# fit_model([1,1])

# gen_test = Generator_V('Test', batch_size, time_frames, False, fold_step)
# gen_test = predict()

# Image('model.png')
# model.summary()

Mounted at /content/drive
     |████████████████████████████████| 51kB 4.2MB/s 
COPYING TRAIN SET...
COPYING VALIDATION SET...
COPYING TEST SET...
COPYING OPENFACE FEATURES...
COPYING LABELS...
DONE


In [ ]:
print("CREATING FOLDS...")
subs = {}
Y = []
clips = []
for split in ["Train", "Validation", "Test"]:
    csv_path = "{}/{}Labels.csv".format(labels_path, split)
    csv = pd.read_csv(csv_path)
    C = np.array(csv['ClipID'])
    Yt = np.array(csv[emotion])
    if len(Y) == 0:
        clips = C
        Y = Yt
    else:
        Y = np.concatenate((Y, Yt), axis=0)
        clips = np.concatenate((clips, C), axis=0)

# transform 0-3 to 0-1
Y = to_categorical((np.floor(Y/2)), 2)
# Y = to_categorical(Y, 5)

# print(Y.shape)
# print(np.unique(np.argmax(Y, axis=1), return_counts=True))

for i in range(len(clips)):
    file_name = clips[i].split(".")[0]
    subject = int(file_name[:6])
    if not subject in subs:
        # subs[subject] = [0,0,0,0,0]
        subs[subject] = [0,0]
    subs[subject] = np.add(subs[subject], Y[i])

# ratio = sum(Y[:,0]) / len(Y)
# print(subs)

folds = []
fold_names = []
for i in subs:
    if len(folds) < n_folds:
        folds.append(subs[i])
        fold_names.append([i])
    else:
        summed = np.sum(folds, axis=1)
        index = list(summed).index(min(summed))
        folds[index] = np.add(folds[index], subs[i])
        fold_names[index].append(i)

# print(folds)
# print(fold_names)
# print(np.sum(folds, axis=1))

print("GETTING FOLDS DATA...")
# build frames array
videos, opface, labels = [], [], []
for i in range(len(fold_names)):
    videos.append({})
    opface.append({})
    labels.append({})

fps = 15
skip = int(round(interval * fps / time_frames, 0))

for split in ["Train", "Validation", "Test"]:
    csv_path = "{}/{}Labels.csv".format(labels_path, split)
    csv = pd.read_csv(csv_path)
    clips = csv['ClipID']
    Y = np.floor(csv[emotion]/2)

    for Yi in range(len(clips)):
        file_name = clips[Yi].split(".")[0]
        subject = file_name[:6]

        for f in range(n_folds):
            if int(subject) in fold_names[f] and Y[Yi] <= 3:
                if subject not in labels[f]:
                    labels[f][subject] = []
                    videos[f][subject] = []
                    opface[f][subject] = []
                # print("{}/{}".format(split, subject))
                # get aligned faces file names into files array
                dir_path = "{}/{}/{}/{}_aligned".format(features_path, split, subject, file_name)
                of_path = "of_{}/{}/{}/{}.csv".format(features_path, split, subject, file_name)
                files, filesof = [], []

                if os.path.isfile(of_path):
                    csv_of = pd.read_csv(of_path)
                    # rows = csv_of.iloc[:,5:]
                    rows = csv_of[[' gaze_0_x',' gaze_0_y',' gaze_0_z',' gaze_1_x',' gaze_1_y',' gaze_1_z',' gaze_angle_x',' gaze_angle_y',' pose_Tx',' pose_Ty',' pose_Tz',' pose_Rx',' pose_Ry',' pose_Rz',' AU01_r',' AU02_r',' AU04_r',' AU05_r',' AU06_r',' AU07_r',' AU09_r',' AU10_r',' AU12_r',' AU14_r',' AU15_r',' AU17_r',' AU20_r',' AU23_r',' AU25_r',' AU26_r',' AU45_r',' AU01_c',' AU02_c',' AU04_c',' AU05_c',' AU06_c',' AU07_c',' AU09_c',' AU10_c',' AU12_c',' AU14_c',' AU15_c',' AU17_c',' AU20_c',' AU23_c',' AU25_c',' AU26_c',' AU28_c',' AU45_c']]
                    min_max_scaler = MinMaxScaler()
                    rows = min_max_scaler.fit_transform(rows)
                
                for i in range(1,350):
                    image_path = "{}/frame_det_00_000{:03d}.jpg".format(dir_path, i)
                    if os.path.isfile(image_path) and os.path.isfile(of_path):
                        files.append(image_path)
                        filesof.append(rows[i-1])

                # append only the parts relative to the video section
                for i in range(0, (len(files) - time_frames*skip) // skip + 1, np.max([1, int(stride * time_frames)])):
                    temp, tpof = [], []
                    for j in range(i*skip, (i+time_frames)*skip, skip):
                        temp.append(files[j])
                        tpof.append(filesof[j])
                    videos[f][subject].append(temp)
                    opface[f][subject].append(tpof)
                    labels[f][subject].append(Y[Yi])



CREATING FOLDS...
GETTING FOLDS DATA...


In [ ]:
# print(len(labels[0]), labels[0])
a,t = 0,0
for fold in labels:
    for sub in fold:
        # if len(fold[sub]) < 30:
        #     print(fold[sub])
        if 0 in fold[sub]:
            print(fold[sub])
            a = a + 1
        t = t + 1
            # del fold[sub]

print(a/t)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,